In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd


Patients_train_path = "/content/drive/MyDrive/Iqram Sir/PK_PD/Patients_train.csv"
Patients_test_path = "/content/drive/MyDrive/Iqram Sir/PK_PD/Patients_test.csv"


Patients_train = pd.read_csv(Patients_train_path, index_col=0)
Patients_test = pd.read_csv(Patients_test_path, index_col=0)

# %% Undersample data

step = 60  # Undersampling step


Patients_test_full = Patients_test.copy()

Patients_train_BIS = Patients_train[Patients_train['full_BIS'] == 0]
Patients_test_BIS = Patients_test[Patients_test['full_BIS'] == 0]
Patients_train_MAP = Patients_train[Patients_train['full_MAP'] == 0]
Patients_test_MAP = Patients_test[Patients_test['full_MAP'] == 0]

Patients_train_BIS = Patients_train_BIS[::step]
Patients_test_BIS = Patients_test_BIS[::step]
Patients_train_MAP = Patients_train_MAP[::step]
Patients_test_MAP = Patients_test_MAP[::step]



feature = 'All'
cov = ['age', 'sex', 'height', 'weight']
Ce_bis_eleveld = ['Ce_Prop_Eleveld', 'Ce_Rem_Eleveld']
Ce_map_eleveld = ['Ce_Prop_MAP_Eleveld', 'Ce_Rem_MAP_Eleveld']
Cplasma_eleveld = ['Cp_Prop_Eleveld', 'Cp_Rem_Eleveld']


# feat_A/ input features
if feature == 'All':
    X_col = cov + ['bmi', 'lbm', 'mean_HR'] + Ce_map_eleveld + Ce_bis_eleveld + Cplasma_eleveld
elif feature == '-bmi':
    X_col = cov + ['lbm', 'MAP_base_case', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
elif feature == '-lbm':
    X_col = cov + ['bmi', 'MAP_base_case', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
elif feature == '-map':
    X_col = cov + ['bmi', 'lbm', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
elif feature == '-hr':
    X_col = cov + ['bmi', 'lbm', 'MAP_base_case'] + Ce_bis_eleveld + Ce_map_eleveld + Cplasma_eleveld
elif feature == '-Cplasma':
    X_col = cov + ['bmi', 'lbm', 'MAP_base_case', 'mean_HR'] + Ce_bis_eleveld + Ce_map_eleveld
elif feature == '-Cmap':
    X_col = cov + ['bmi', 'lbm', 'MAP_base_case', 'mean_HR'] + Ce_bis_eleveld + Cplasma_eleveld
elif feature == '-Cbis':
    X_col = cov + ['bmi', 'lbm', 'MAP_base_case', 'mean_HR'] + Ce_map_eleveld + Cplasma_eleveld


Patients_train_BIS = Patients_train_BIS.dropna().reset_index(drop=True)
Patients_test_BIS = Patients_test_BIS.dropna().reset_index(drop=True)
Patients_train_MAP = Patients_train_MAP.dropna().reset_index(drop=True)
Patients_test_MAP = Patients_test_MAP.dropna().reset_index(drop=True)




X_train_BIS = Patients_train_BIS[X_col]
y_train_BIS = Patients_train_BIS['BIS']

X_test_BIS = Patients_test_BIS[X_col]
y_test_BIS = Patients_test_BIS['BIS']




X_train_MAP = Patients_train_MAP[X_col]
y_train_MAP = Patients_train_MAP['MAP']

X_test_MAP = Patients_test_MAP[X_col]
y_test_MAP = Patients_test_MAP['MAP']

X_train_BIS.shape,y_train_BIS.shape,X_test_BIS.shape,y_test_BIS.shape

((19055, 13), (19055,), (7932, 13), (7932,))

In [4]:
X_train,X_test,y_train,y_test = X_train_BIS,X_test_BIS,y_train_BIS,y_test_BIS
X = X_train.to_numpy()
y = y_train.to_numpy()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train.shape[1],), return_sequences=True),
    BatchNormalization(),
    LSTM(32, activation='relu', return_sequences=False),
    BatchNormalization(),
    Dense(1, activation='linear')  # Assuming it's a regression task; change activation for classification
])

model.compile(optimizer='adam', loss='mean_squared_error')  # Adjust loss based on your task

history = model.fit(X_train_reshaped, y_train, epochs=20, batch_size=32, validation_split=0.2)  # Adjust batch size and validation split


NameError: name 'train_test_split' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Reshape, Flatten
from tensorflow.keras.applications import DenseNet121
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np




# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model 1: Feedforward Neural Network
model_ffnn = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='linear')
])

model_ffnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Model 2: Convolutional Neural Network (CNN)
model_cnn = Sequential([
    Reshape((13, 1), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')
])

model_cnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

